# Уравнение переноса

Перенос - один из важнейших в атмосфере. Перемещаясь воздух переносит с собой свои характеристики - температуру, влажность, концентрации пыли и других веществ, даже свою скорость!

Рассмотрим кубик воздуха, который движется "по воле ветра". Пусть в этом кубике концентрация озона (например, но может быть любая) $u$ моль/моль воздуха. Обратите внимание на размерность концентрации - она безразмерна, это количество молей озона в кубике, поделенное на количество молей воздуха в кубике.

Предположим, что сейчас никаких химических реакций нет и количество озона в кубике не меняется. Количество воздуха тоже не меняется (почему, кстати?). Это значит, что концентрация $u$ в кубике постоянная. Но сам кубик двигается, поэтому поле конентрации меняется со временем.

Выведем уравнение для изменения концентрации. Пусть сейчас концентрация $u = u(t,x,y,z)$, и скорость ветра $v_x, v_y, v_z$. Через время $\Delta t$ в точку $x,y,z$ прилетит другой кубик воздуха из точки $x-v_x\Delta t$, $y-v_y\Delta t$, $z-v_z\Delta t$ (тут точность до $\Delta t^2$). Получается, 
\begin{equation}
u(t+\Delta t,x,y,z) = u(t,x-v_x\Delta t, y-v_y\Delta t, z-v_z\Delta t).
\end{equation}

Вычтем $u(t,x,y,z)$ слева и справа и устремим $\Delta t$ к нулю, чтобы перейти к производным. Получается 
\begin{equation}
\frac{\partial u}{\partial t} = -v_x\frac{\partial u}{\partial x}
 -v_y\frac{\partial u}{\partial y}
  -v_z\frac{\partial u}{\partial z}.
\end{equation}.

Когда есть какие-то источники или стоки переносимой величины, то они входя в правую часть уравнения:
\begin{equation}
\frac{\partial u}{\partial t} +v_x\frac{\partial u}{\partial x}
 +v_y\frac{\partial u}{\partial y}
  +v_z\frac{\partial u}{\partial z} = F(u).
\end{equation}.

## Одномерное уравнение переноса
Для первичного тестирования численных схем уравнение переноса упрощают. Пусть ветер дует только вдоль оси $x$ и постоянен, $v_x = c$, предположим также периодичские граничные условия:
\begin{align}
&\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0, \ x \in [0,2\pi), \ c>0, \\
&u(t,0) = u(t,2\pi).
\end{align}

Это самое простое уравнение в частных производных. Проще не бывает:-).

Для одномерного уравнения переноса с постоянной скоростью есть аналитическое решение. Если $u(t=0,x) = u_0(x)$, то
\begin{equation}
u(t,x) = u_0(x-ct)
\end{equation}

### Упражение 1
Пусть $x\in[-\pi,\pi]$, $u_0(x) = \sin(x)$, $c=1$. Нарисуйте аналитическое решение в моменты времени $t=0.1\pi, 0.5\pi,\pi,2\pi$. Сделайте тоже самое для функции
\begin{align}
u_0(x) &= 1,\quad \lvert x\rvert < \pi/2 \\
u_o(x) &= 0,\quad \text{иначе}
\end{align}

## Простейшая численная схема

Рассмотрим простейший вариант для аппроксимации этого уравнения. 
Для аппроксимации по времени можем воспользоваться явным методом Эйлера:
$$\frac{u^{n+1}(x)-u^n(x)}{\Delta t} + с \left(\frac{\partial u}{\partial x}\right)^n=0. $$

Для аппроксимации по пространству можно использовать:
\begin{align}
\left(\frac{\partial u}{\partial x}\right)^n \approx \frac{u^n_{i}-u^n_{i-1}}{\Delta x}.
\end{align}
Собирая все вместе, получаем схему "левый явный уголог"
\begin{align}
\frac{u^{n+1}_i-u^n_i}{\Delta t} + c \frac{u^n_{i}-u^n_{i-1}}{\Delta x} = 0.
\end{align}

Перейдем к программной реализации. Будем использовать равномерную сетку на периодическом отрезке $x \in [0,L)$. Количество узлов сетки $N_x+1$, шаг сетки $\Delta x = L/N_x$, узлы сетки $x_i = (i-1)\Delta x$, $i=0 \dots N_x$. Решаем уравнение при $t \in [0, T]$, количество шагов по времени $N_t$, $t_n = (n-1) \Delta t$, $n=0 \dots N_t$.

In [1]:
# Подключаем необходимые пакеты
import numpy as np    
import matplotlib.pyplot  as plt
from matplotlib import animation
%matplotlib notebook

Реализуем процедуру для вычисления производной по пространству.

In [2]:
def leftDifference(f, dx):
    diff_f = np.empty_like(f)
    for i in range(1, f.size):
        if i == 0:
            diff_f[i] = (f[1]-f[-2])/(2*dx)
        else:
            diff_f[i] = (f[i]-f[i-1])/dx
    diff_f[f.size-1] = diff_f[0]
    return diff_f

Функция для выполнения шага по времени при помощи явного метода Эйлера.

In [3]:
def explicitEulerStep(state, func, dt):
    return state + dt*func(state) 

Функция для решения уравнения переноса. 

In [4]:
def solveAdvection(u0, timeMethod, spaceMethod, nx = 100, nt = 200, L = 1.0, T = 1.0, c = 1.0):
    
    # параметры пространственной сетки
    dx = L / nx
    x = np.arange(0, nx + 1) * dx
    # параметры временной сетки
    dt = T / nt
    t = np.arange(0, nt + 1) * dt
    # число Куранта
    CFL = c * dt / dx
    print(f"Число Куранта CFL = {CFL}")
    
    # инициализируем массивы для хранения численного и точного решения во все моменты времени
    u = np.zeros((nt+1, nx+1))
    uExact = np.zeros((nt+1, nx+1))
    # задаем решение в начальный момент времени
    u[0,:] = u0(x)
    uExact[0,:] = u0(x)
    # цикл по времени
    for k in range(nt):
        u[k+1,:] = timeMethod(u[k,:], lambda u: -c*spaceMethod(u, dx), dt)
        uExact[k+1, :] = u0((x-c*t[k+1])%np.max(x))
        
    return u, uExact, x, t

Функции для задания начального профиля.

In [5]:
def gaussianHill(x, mean = 0.5, sigma = 10):
    return np.exp(-sigma**2*(x-mean)**2)

Функция для отрисовки численного и точного решения

In [6]:
def animateAdvection(u, uExact, x, t, animSpeed = 1):

    fig, ax = plt.subplots()
    line1, = ax.plot(x, u[0,:], label = "Numerical Solution")
    line2, = ax.plot(x,uExact[0,:], label = "Exact Solution") 
    lines = (line1, line2)
    
    fig.legend(loc = 8, ncol = 2) 
    fig.tight_layout()
    fig.subplots_adjust(bottom=0.15)
    
    height = np.max(np.abs(u[0,:]))
    ax.set_ylim(np.min(u[0, :]) - 0.2 * height, np.max(u[0, :]) + 0.2 * height)

    time_template = 'time = %.2fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    def animate(n):
        nAnim = animSpeed * n
        lines[0].set_ydata(u[nAnim,:])
        lines[1].set_ydata(uExact[nAnim,:])
        time_text.set_text(time_template % (t[nAnim]))
        return lines, time_text

    anim = animation.FuncAnimation(fig, animate, frames=len(t), interval=1, repeat = False, blit=True)
    plt.show()

    return anim

In [7]:
u, uExact, x, t = solveAdvection(gaussianHill, explicitEulerStep, leftDifference, nx = 100, nt = 200)
anim = animateAdvection(u, uExact, x, t)
anim.save('anim.gif')

Число Куранта CFL = 0.5


<IPython.core.display.Javascript object>

# Упражнения/задания

### Упражнения 2. Смотрим разные сочетания численных схем

Попробуйте решить уравнение переноса с помощью предложенных сочетаний схем по времени и пространству.
Какие-то сочетания заведомо не рабочие :-). Сохраните рисунки, охарактеризуйте для себя отклонение каждой схемы от точного решения.

### Упражнение 2.1
Запустите схему переноса РК4 + левая разность

### Упражнение 2.2
Запустите схему переноса явный Эйлер + центральная разность второго порядка.

### Упражнение 2.3
Запустите схему переноса явный Эйлер + правая разность.

### Упражнение 2.4
Запустите схему переноса РК4 + центральная разность второго порядка.

### Упражнение 2.5
Запустите схему переноса РК4 + центральная разность 4-го порядка.

### Упражнение 2.6
Схема РК4 и аппроксимация производной
\begin{equation}
\frac{\partial u}{\partial x}(x) \approx \frac{3u(x+h)+10u(x)-18u(x-h)+6u(x-2h)-u(x-3h)}{12h}
\end{equation}

### Упражнение 2.7
Повторите все тоже самое для начального условия типа "Кирпич"
\begin{align}
u_0(x) &= 1,\quad \lvert x\rvert < \pi/2 \\
u_o(x) &= 0,\quad \text{иначе}
\end{align}
Чем схема с левой разностью лучше более центральных разностей 2-го 4-го порядков?

In [ ]:
def RK4(state, f,  dt):
    k1 = f(state)
    k2 = f(state + dt/2 * k1)
    k3 = f(state + dt/2 * k2)
    k4 = f(state + dt * k3)
    return state + dt/6 * (k1 + 2*k2 + 2*k3 + k4)


def leftDifference(u, dx):
    diff_f = np.empty_like(u)
    for i in range(1, u.size):
        diff_f[i] = (u[i] - u[i - 1]) / dx
    diff_f[0] = diff_f[-1] 
    return diff_f

def rightDifference(u, dx):
    diff_f = np.empty_like(u)
    for i in range(1, u.size):
        diff_f[i] = (u[i+1] - u[i]) / dx
    diff_f[-1] = diff_f[0] 
    return diff_f

def centralDifference2(u, dx):
    N = len(u)
    diff = np.empty_like(u)
    for i in range(N):
        diff[i] = (u[(i+1) % N] - u[(i-1) % N]) / (2 * dx)
    diff[0] = (u[1] - u[-2]) / (2 * dx) 
    diff[-1] = diff[0]
    return diff

def centralDifference4(u, dx):
    N = len(u)
    diff = np.empty_like(u)
    for i in range(N):
        diff[i] = (u[(i-2) % N] - 8*u[(i-1) % N] + 8*u[(i+1) % N] - u[(i+2) % N])/(12 * dx)
    diff[0] = (-u[2] + 8 * u[1] - 8 * u[-2] + u[-3]) / (12 * dx)
    diff[1] = (-u[3] + 8 * u[2] - 8 * u[0] + u[-2]) / (12 * dx)
    diff[-1] = diff[0]
    diff[-2] = (-u[1] + 8 * u[0] - 8 * u[-3] + u[-4]) / (12 * dx)
    return diff

def customDiff(u, dx):
    n = len(u)
    diff = np.empty_like(u)
    for i in range(n):
        diff[i] = (3*u[(i+1) % n] + 10*u[i] - 18*u[(i-1) % n] + 6*u[(i-2) % n] - u[(i-3) % n])/(12 * dx)
    diff[0] = (3 * u[1] + 10 * u[0] - 18 * u[-2] + 6 * u[-3] - f[-4]) / (12 * dx)
    diff[1] = (3 * u[2] + 10 * u[1] - 18 * u[0] + 6 * u[-2] - f[-3]) / (12 * dx)
    diff[2] = (3 * u[3] + 10 * u[2] - 18 * u[1] + 6 * u[0] - f[-2]) / (12 * dx)
    diff[-1] = diff[0]
    return diff

def gaussianHill(x, mean=0.5, sigma=10):
    return np.exp(-sigma**2 * (x - mean)**2)

def brick(x):
    center = width = np.pi
    return np.where(np.abs(x - center) < width/2, 1.0, 0.0)


def solve(u0, timeMethod, spaceMethod, nx = 100, nt = 200, L = 1.0, T = 1.0, c = 1.0):
    dx = L / nx
    x = np.linspace(0, L, nx + 1)
    dt = T / nt
    t = np.linspace(0, T, nt + 1)
    CFL = c * dt / dx
    # print(f"Число Куранта CFL = {CFL}")
    
    u = np.zeros((nt + 1, nx + 1))
    uExact = np.zeros((nt + 1, nx + 1))
    
    u[0, :] = u0(x)
    uExact[0, :] = u0(x)
    
    for k in range(nt):
        u[k + 1, :] = timeMethod(u[k, :], lambda u_val: -c * spaceMethod(u_val, dx), dt)
        uExact[k + 1, :] = u0((x - c * t[k + 1]) % L)
        
    return u, uExact, x, t

schemes = {
    '2.1: RK4 + левая разность'           : (RK4, leftDifference),
    '2.2: Euler + центральная (2-ого)'     : (explicitEulerStep, centralDifference2),
    '2.3: Euler + правая разность'         : (explicitEulerStep, rightDifference),
    '2.4: RK4 + центральная (2-ого)'       : (RK4, centralDifference2),
    '2.5: RK4 + центральная (4-ого)'       : (RK4, centralDifference4),
    '2.6: RK4 + своя формула'              : (RK4, customDiff)
}


def run_experiment(u0, L, scheme_dict, title_suffix):

    n_schemes = len(scheme_dict)
    ncols = 2
    nrows = int(np.ceil(n_schemes / ncols))
    
    plt.figure(figsize=(30, 30))
    plt.suptitle("Результаты решения уравнения переноса: " + title_suffix, fontsize=14)
    
    for i, (scheme_name, (timeMethod, spaceMethod)) in enumerate(scheme_dict.items()):
        u, uExact, x, t = solve(u0, timeMethod, spaceMethod, nx=100, nt=200, L=L, T=1.0, c=1.0)
        anim = animateAdvection(u, uExact, x, t)
        anim.save(f'{i} {title_suffix}.gif')
    
    
    



print("Эксперимент: начальное условие – Гауссова куча (gaussianHill)")
run_experiment(gaussianHill, L=1.0, scheme_dict=schemes, title_suffix="Начальное условие: Гауссова куча, L = 1.0")

print("Эксперимент: начальное условие – Кирпич (brick)")
run_experiment(brick, L=2*np.pi, scheme_dict=schemes, title_suffix="Начальное условие: Кирпич, L = 2π")


### Упражнение 3
Для схем РК4+левая разность, РК4+центральная разность 2-го и 4-го порядка постройте графики уменьшения ошибки при уменьшении шага по времени (в лог масштабе). Какая схема самая точная? Используйте начальное условие - гауссиану.

**NB** Уменьшая шаг сетки по пространству, уменьшайте и шаг по времени так, чтобы число Куранта $C = \frac{c\Delta t}{h}$ оставалось постоянно.

In [13]:
def get_err(u0, timeMethod, spaceMethod, nx, CFL, T=1.0, L=1.0, c=1.0):
    dx = L / nx
    dt = CFL * dx / c # число куранта - константа
    nt = int(np.round(T / dt))
    u, uExact, x, t = solve(u0, timeMethod, spaceMethod, nx=nx, nt=nt, L=L, T=T, c=c)
    err = np.sqrt(np.sqrt(np.mean((u - uExact)**2)))
    return dt, err

nx_values = [int(i) for i in range(100, 1500, 200)]

CFL = 0.5
T = 1.0
L = 1.0
c = 1.0

schemes = {
    'RK4 + левая разность': (RK4, leftDifference),
    'RK4 + центральная (2-ого)': (RK4, centralDifference2),
    'RK4 + центральная (4-ого)': (RK4, centralDifference4)
}

errors = {scheme: [] for scheme in schemes}
dts = {scheme: [] for scheme in schemes}


for nx in nx_values:
    for scheme_name, (timeMethod, spaceMethod) in schemes.items():
        dt, err = get_err(gaussianHill, timeMethod, spaceMethod, nx, CFL, T, L, c)
        errors[scheme_name].append(err)
        dts[scheme_name].append(dt)
    
    # print(f'{scheme_name}  ошибка {errors[scheme_name]} шаг {dts[scheme_name]}')

plt.figure(figsize=(12,10))
for scheme in schemes:
    plt.loglog(dts[scheme], errors[scheme], marker='o', label=scheme)
plt.xlabel("Шаг по времени dt")
plt.ylabel("L2-норма ошибки")
plt.title("Сходимость для различных схем (начальное условие – гауссиана)")
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()
plt.savefig('error_convergence2.png')




<IPython.core.display.Javascript object>

### Упражнение 4
Возьмите сочетание схем, которое вам больше всех нравится и начальное условие Кирпич. Увеличивайте шаг по времени, пока решение не станет неустойчивым. Какое число Куранта критическое?

In [10]:
nx = 200
L = 2 * np.pi
c = 1.0
T = 1.0

cfl_values = np.linspace(2, 2.5, 1024)
critical_cfl = -1

for CFL in cfl_values:
    dx = L / nx
    dt = CFL * dx / c
    nt = int(np.round(T / dt))
    u, uExact, x, t = solve(brick, RK4, centralDifference4, nx=nx, nt=nt, L=L, T=T, c=c)
    max_u = np.max(np.abs(u[-1, :]))
    print(f"CFL = {CFL:.4f}, nt = {nt:4d}, max(|u|) = {max_u:.2f}")
    if max_u > 5:
        critical_cfl = CFL
        break

print(f"Критическое число Куранта: {critical_cfl:.4f}")

plt.figure(figsize=(10,12))
plt.plot(x, u[-1, :], label=f'Численное решение, CFL = {critical_cfl:.5f}')
plt.plot(x, uExact[-1, :], '--', label='Точное решение')
plt.xlabel("x")
plt.ylabel("u")
plt.title("Результат численного решения (устройство упражнения 4)")
plt.legend()
plt.grid(True)
plt.show()

plt.savefig('critical_cfl.png')


CFL = 2.0000, nt =   16, max(|u|) = 1.10
CFL = 2.0005, nt =   16, max(|u|) = 1.10
CFL = 2.0010, nt =   16, max(|u|) = 1.10
CFL = 2.0015, nt =   16, max(|u|) = 1.10
CFL = 2.0020, nt =   16, max(|u|) = 1.10
CFL = 2.0024, nt =   16, max(|u|) = 1.10
CFL = 2.0029, nt =   16, max(|u|) = 1.10
CFL = 2.0034, nt =   16, max(|u|) = 1.10
CFL = 2.0039, nt =   16, max(|u|) = 1.10
CFL = 2.0044, nt =   16, max(|u|) = 1.10
CFL = 2.0049, nt =   16, max(|u|) = 1.10
CFL = 2.0054, nt =   16, max(|u|) = 1.10
CFL = 2.0059, nt =   16, max(|u|) = 1.10
CFL = 2.0064, nt =   16, max(|u|) = 1.10
CFL = 2.0068, nt =   16, max(|u|) = 1.10
CFL = 2.0073, nt =   16, max(|u|) = 1.10
CFL = 2.0078, nt =   16, max(|u|) = 1.10
CFL = 2.0083, nt =   16, max(|u|) = 1.10
CFL = 2.0088, nt =   16, max(|u|) = 1.10
CFL = 2.0093, nt =   16, max(|u|) = 1.10
CFL = 2.0098, nt =   16, max(|u|) = 1.10
CFL = 2.0103, nt =   16, max(|u|) = 1.10
CFL = 2.0108, nt =   16, max(|u|) = 1.10
CFL = 2.0112, nt =   16, max(|u|) = 1.10
CFL = 2.0117, nt

<IPython.core.display.Javascript object>

# Фазовая и амплитудная ошибки

Пусть мы используем идеальную схему интегрирования по времени для уравнения переноса. Рассмотрим как переносится начальное условие $A_0\exp\{ikx\}$. Функция такого типа - собственная как для аналитической $\partial/\partial x$, так и для конечных разностей. Важно, что экспонента - элемент Фурье базиса, а все что угодно периодическое раскладывается в ряд Фурье. Мы сейчас будем изучать как себя ведет одна Фурье гармоника.

**Для аналитической** $\partial/\partial x$
\begin{align}
& \frac{\partial }{\partial t}A(t)\exp\{ikx\} = 
-c\frac{\partial }{\partial x}A(t)\exp\{ikx\} \\
&\frac{\partial }{\partial t}A(t) =-ikcA(t) \\
& A = \exp\{-ikct\}
\end{align}
аналитическое решение для такого начального условия $f(x,t) = A_0\exp\{ik(x-ct)\}$

**Для конечно-разностной производной** $D$:
\begin{equation}
D\exp\{ikx\} = (k_r+i k_{im})\exp{ikx}
\end{equation}
Решение численной схемы получается
\begin{equation}
f = A_0\exp(-ck_r t)\exp\{ikx - ik_{im}c t\}.
\end{equation}

Если у аппроксимации пространственной производной есть действительная часть $k_r$ то решение будет затухать (левая разность, $c>0$) или неограничено расти (правая разность, $c<0$). Мнимая часть собственного числа как правило $k_{im}\le k$. Т.е. численное решение отстает от истинного, причем, чем больше $k$ (длина волны короче), тем хуже.

In [11]:
from functools import lru_cache

@lru_cache(maxsize=None)
def aker(m , n):
    if m == 0:
        return n + 1
    elif m > 0 and n == 0:
        return aker(m - 1, 1)
    elif m > 0 and n > 0:
        return aker(m - 1, aker(m, n - 1))